In [ ]:
!pip freeze | grep scikit-learn

In [1]:
import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/mbici/anaconda3/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mbici/anaconda3/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
year =2022
month = 2

In [6]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
import numpy as np

In [10]:
np.std(y_pred)

5.28140357655334

In [11]:
df['ride_id']= f'{year:04d}/{month:02d}_'+df.index.astype('str')

In [26]:
df_results = pd.concat([df['ride_id'],pd.DataFrame(y_pred)],axis=1)
df_results.rename(columns ={0:"predicted_duration"},inplace= True)
df_results


,ride_id,predicted_duration
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904
...,...,...
2917969,NaN,9.501657
2917970,NaN,28.938670
2917975,NaN,11.729149
2918158,NaN,10.485285


In [27]:
df_results.to_parquet(
    f"{year:04d}_{month:02d}.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

In [28]:
import os

file_stats = os.stat(r"/home/mbici/Desktop/2023/mlops-zoomcamp/cohorts/2023/04-deployment/homework/2022_02.parquet")

In [33]:
print(f"The file size is {round(file_stats.st_size/(1024*1024),2)}MB")

The file size is 57.22MB
